# Lab 15.5: Global Operations and Multi-Currency Support

**Duration:** 13 minutes  
**Difficulty:** Advanced  

## Learning Objectives

In this notebook, you will:
- Create international subsidiaries (UK and Canada)
- Implement regulatory frameworks (FCA, OSFI)
- Build multi-currency exchange systems
- Verify complete multi-line insurance platform

---

## Step 1: Connect to Neo4j

Establish connection and verify reinsurance networks are in place.

In [ ]:
from neo4j import GraphDatabase
import pandas as pd
from datetime import datetime, date
import uuid

# Connect to Neo4j Enterprise instance
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

def run_query(query, parameters=None):
    with driver.session(database="insurance") as session:
        result = session.run(query, parameters)
        return [record.data() for record in result]

# Verify current database state
current_state = run_query("""
MATCH (n) 
RETURN labels(n)[0] AS node_type, count(n) AS count
ORDER BY count DESC
""")

print("Current Database State:")
for record in current_state:
    print(f"  {record['node_type']}: {record['count']} nodes")

## Step 2: Understanding Global Insurance Operations

### Business Context: International Insurance Regulation

**United Kingdom - FCA/PRA Regulation:**
- **FCA (Financial Conduct Authority)**: Regulates conduct and consumer protection
- **PRA (Prudential Regulation Authority)**: Supervises safety and soundness
- **Solvency II Framework**: European capital adequacy directive
  - SCR (Solvency Capital Requirement): 99.5% VaR over 1 year
  - MCR (Minimum Capital Requirement): Absolute floor for capital
  - Own Risk and Solvency Assessment (ORSA): Annual risk evaluation
- **UK GDPR**: Data protection requirements for customer information
- **SMCR (Senior Managers & Certification Regime)**: Personal accountability framework

**Canada - OSFI Regulation:**
- **OSFI (Office of the Superintendent of Financial Institutions)**: Federal regulator
- **Provincial Licensing**: Must be licensed in each province of operation
- **MCCSR (Minimum Continuing Capital and Surplus Requirements)**: Capital adequacy
  - Target ratio: >150% for life insurers
  - Enhanced monitoring at 120-150%
- **DSCR (Dynamic Solvency Testing)**: Forward-looking capital assessment
- **PIPEDA (Personal Information Protection)**: Privacy legislation

**Multi-Currency Operations:**
- All financial reporting consolidated to USD
- Local statutory reporting in local currency (GBP, CAD)
- Foreign exchange risk management
- Transfer pricing between subsidiaries
- Intercompany reinsurance transactions

In [ ]:
# Create international subsidiaries and regulatory frameworks
global_operations_query = """
// Create International Subsidiaries
CREATE (uk_subsidiary:Subsidiary:LegalEntity {
  id: randomUUID(),
  entity_id: "SUB-UK-001",
  entity_name: "Apex Insurance UK Limited",
  entity_type: "Private Limited Company",
  incorporation_country: "United Kingdom",
  incorporation_date: date("2018-03-15"),
  
  // UK Registration
  company_number: "11234567",
  registered_office: "Level 25, The Leadenhall Building, 122 Leadenhall Street, London EC3V 4AB",
  vat_number: "GB987654321",
  
  // Regulatory details
  fca_registration: "765432",
  regulatory_status: "Authorized",
  prudential_regulation: "PRA Supervised",
  
  // Financial information
  share_capital: 15000000.00,
  solvency_ratio: 1.85,
  mcr_coverage: 2.45,
  scr_coverage: 1.65,
  
  // Business metrics
  gross_written_premium: 45000000.00,
  employee_count: 125,
  branch_locations: ["London", "Manchester", "Edinburgh"],
  
  created_at: datetime(),
  created_by: "global_operations",
  version: 1
})

CREATE (canada_subsidiary:Subsidiary:LegalEntity {
  id: randomUUID(),
  entity_id: "SUB-CA-001",
  entity_name: "Apex Insurance Canada Inc.",
  entity_type: "Federal Corporation",
  incorporation_country: "Canada",
  incorporation_date: date("2019-08-22"),
  
  // Canadian Registration
  corporation_number: "987654321",
  registered_office: "Suite 4000, 421 7th Avenue SW, Calgary, AB T2P 4K9",
  business_number: "123456789RC0001",
  
  // Regulatory details
  osfi_registration: "INS-456789",
  regulatory_status: "Licensed",
  provincial_licenses: ["AB", "ON", "BC", "QC"],
  
  // Financial information (CAD)
  share_capital: 20000000.00,
  mccsr_ratio: 185.0,
  surplus: 12500000.00,
  
  // Business metrics
  gross_written_premium: 32000000.00,
  employee_count: 85,
  branch_locations: ["Calgary", "Toronto", "Vancouver"],
  
  created_at: datetime(),
  created_by: "global_operations",
  version: 1
})

// Create Regulatory Frameworks
CREATE (uk_regulation:RegulatoryFramework {
  id: randomUUID(),
  framework_id: "REG-UK-001",
  framework_name: "UK Solvency II Framework",
  jurisdiction: "United Kingdom",
  regulatory_body: "Financial Conduct Authority (FCA)",
  
  // Regulatory requirements
  solvency_requirements: "Solvency II Directive",
  capital_requirements: "SCR and MCR",
  reporting_frequency: "Quarterly",
  audit_requirements: "Annual External Audit",
  
  // Compliance requirements
  conduct_rules: "Senior Managers & Certification Regime",
  consumer_protection: "ICOBS and COBS",
  data_protection: "UK GDPR",
  
  // Reporting obligations
  rsrp_reporting: "Required",
  orsa_requirement: "Annual",
  pillar_3_disclosure: "Annual",
  
  effective_date: date("2016-01-01"),
  last_updated: date("2024-01-01"),
  
  created_at: datetime(),
  created_by: "regulatory_compliance",
  version: 1
})

CREATE (canada_regulation:RegulatoryFramework {
  id: randomUUID(),
  framework_id: "REG-CA-001",
  framework_name: "Canadian Insurance Regulatory Framework",
  jurisdiction: "Canada",
  regulatory_body: "Office of the Superintendent of Financial Institutions (OSFI)",
  
  // Regulatory requirements
  solvency_requirements: "OSFI Capital Requirements",
  capital_requirements: "MCCSR Guidelines",
  reporting_frequency: "Quarterly",
  audit_requirements: "Annual External Audit",
  
  // Compliance requirements
  conduct_rules: "Market Conduct Guidelines",
  consumer_protection: "Fair Treatment of Customers",
  privacy_legislation: "PIPEDA",
  
  // Reporting obligations
  annual_statement: "Required",
  dscr_reporting: "Dynamic Solvency Testing",
  capital_adequacy: "MCCSR Ratio > 150%",
  
  effective_date: date("2018-01-01"),
  last_updated: date("2024-01-01"),
  
  created_at: datetime(),
  created_by: "regulatory_compliance",
  version: 1
})

RETURN uk_subsidiary.entity_name AS uk_entity,
       canada_subsidiary.entity_name AS canada_entity,
       uk_regulation.framework_name AS uk_framework,
       canada_regulation.framework_name AS canada_framework
"""

global_results = run_query(global_operations_query)
print("Global Operations Infrastructure Created:")
for record in global_results:
    print(f"  UK Subsidiary: {record['uk_entity']}")
    print(f"  Canada Subsidiary: {record['canada_entity']}")
    print(f"  UK Regulatory Framework: {record['uk_framework']}")
    print(f"  Canada Regulatory Framework: {record['canada_framework']}")

## Step 3: Create Multi-Currency Exchange System

### Business Context: Currency Management

**Exchange Rate Types:**
- **Spot Rate**: Current market exchange rate for immediate settlement
- **Forward Rate**: Agreed exchange rate for future date (hedging)
- **Average Rate**: Used for income statement translation
- **Period-End Rate**: Used for balance sheet translation

**Bid-Ask Spread:**
- **Bid Rate**: Rate at which bank buys foreign currency
- **Ask Rate**: Rate at which bank sells foreign currency
- **Spread**: Difference represents bank's profit margin

**Financial Reporting:**
- Local currency financials for statutory reporting
- USD consolidation for group reporting
- Foreign exchange gains/losses recognized
- Hedging strategies to minimize FX volatility

In [ ]:
# Create currency exchange framework
currency_exchange_query = """
// Create Currency Exchange Framework
CREATE (usd_base:CurrencyExchange {
  id: randomUUID(),
  exchange_id: "CUR-USD-BASE",
  base_currency: "USD",
  quote_currency: "USD",
  exchange_rate: 1.0000,
  rate_date: date(),
  rate_type: "Base",
  
  // Rate information
  provider: "Internal",
  rate_source: "Base Currency",
  last_updated: datetime(),
  
  created_at: datetime(),
  created_by: "treasury_operations",
  version: 1
})

CREATE (gbp_exchange:CurrencyExchange {
  id: randomUUID(),
  exchange_id: "CUR-GBP-USD",
  base_currency: "GBP",
  quote_currency: "USD",
  exchange_rate: 1.2675,
  rate_date: date(),
  rate_type: "Spot",
  
  // Rate information
  bid_rate: 1.2670,
  ask_rate: 1.2680,
  provider: "Reuters",
  rate_source: "Market Data",
  last_updated: datetime(),
  
  created_at: datetime(),
  created_by: "treasury_operations",
  version: 1
})

CREATE (cad_exchange:CurrencyExchange {
  id: randomUUID(),
  exchange_id: "CUR-CAD-USD",
  base_currency: "CAD",
  quote_currency: "USD",
  exchange_rate: 0.7385,
  rate_date: date(),
  rate_type: "Spot",
  
  // Rate information
  bid_rate: 0.7380,
  ask_rate: 0.7390,
  provider: "Reuters",
  rate_source: "Market Data",
  last_updated: datetime(),
  
  created_at: datetime(),
  created_by: "treasury_operations",
  version: 1
})

RETURN gbp_exchange.base_currency AS gbp_currency,
       gbp_exchange.exchange_rate AS gbp_to_usd,
       cad_exchange.base_currency AS cad_currency,
       cad_exchange.exchange_rate AS cad_to_usd
"""

currency_results = run_query(currency_exchange_query)
print("\nCurrency Exchange Rates Created:")
for record in currency_results:
    print(f"  {record['gbp_currency']}/USD: {record['gbp_to_usd']}")
    print(f"  {record['cad_currency']}/USD: {record['cad_to_usd']}")

## Step 4: Link Subsidiaries to Regulatory Frameworks and Currencies

In [ ]:
# Create subsidiary relationships
subsidiary_relationships_query = """
// Match subsidiaries, regulations, and currencies
MATCH (uk_subsidiary:Subsidiary {entity_id: "SUB-UK-001"})
MATCH (canada_subsidiary:Subsidiary {entity_id: "SUB-CA-001"})
MATCH (uk_regulation:RegulatoryFramework {framework_id: "REG-UK-001"})
MATCH (canada_regulation:RegulatoryFramework {framework_id: "REG-CA-001"})
MATCH (gbp_exchange:CurrencyExchange {exchange_id: "CUR-GBP-USD"})
MATCH (cad_exchange:CurrencyExchange {exchange_id: "CUR-CAD-USD"})

// Create Subsidiary Relationships
CREATE (uk_subsidiary)-[:OPERATES_UNDER {
  license_date: date("2018-05-01"),
  license_number: "765432",
  compliance_status: "Current"
}]->(uk_regulation)

CREATE (canada_subsidiary)-[:OPERATES_UNDER {
  license_date: date("2019-10-15"),
  license_number: "INS-456789",
  compliance_status: "Current"
}]->(canada_regulation)

CREATE (uk_subsidiary)-[:USES_CURRENCY {
  primary_currency: true,
  conversion_frequency: "Daily"
}]->(gbp_exchange)

CREATE (canada_subsidiary)-[:USES_CURRENCY {
  primary_currency: true,
  conversion_frequency: "Daily"
}]->(cad_exchange)

RETURN count(*) AS relationships_created
"""

relationship_results = run_query(subsidiary_relationships_query)
print("\nSubsidiary Relationships Established:")
for record in relationship_results:
    print(f"  Total Relationships: {record['relationships_created']}")

## Step 5: Global Operations Analysis Queries

In [ ]:
# Query 1: Global subsidiary overview
subsidiary_overview_query = """
MATCH (sub:Subsidiary)-[:OPERATES_UNDER]->(reg:RegulatoryFramework)
MATCH (sub)-[:USES_CURRENCY]->(cur:CurrencyExchange)
RETURN sub.entity_name AS subsidiary,
       sub.incorporation_country AS country,
       sub.regulatory_status AS status,
       reg.regulatory_body AS regulator,
       sub.employee_count AS employees,
       sub.gross_written_premium AS gwp_local,
       cur.base_currency AS currency,
       cur.exchange_rate AS fx_rate,
       round(sub.gross_written_premium * cur.exchange_rate, 2) AS gwp_usd
ORDER BY gwp_usd DESC
"""

subsidiary_df = pd.DataFrame(run_query(subsidiary_overview_query))
print("\nGlobal Subsidiary Overview:")
print(subsidiary_df.to_string(index=False))

In [ ]:
# Query 2: Regulatory compliance summary
regulatory_compliance_query = """
MATCH (sub:Subsidiary)-[r:OPERATES_UNDER]->(reg:RegulatoryFramework)
RETURN sub.entity_name AS subsidiary,
       reg.framework_name AS framework,
       reg.jurisdiction AS jurisdiction,
       reg.solvency_requirements AS solvency,
       reg.reporting_frequency AS reporting,
       r.compliance_status AS status,
       sub.solvency_ratio AS solvency_ratio,
       CASE 
         WHEN sub.solvency_ratio >= 1.5 THEN 'Excellent'
         WHEN sub.solvency_ratio >= 1.2 THEN 'Good'
         WHEN sub.solvency_ratio >= 1.0 THEN 'Adequate'
         ELSE 'Requires Attention'
       END AS capital_position
"""

compliance_df = pd.DataFrame(run_query(regulatory_compliance_query))
print("\nRegulatory Compliance Summary:")
print(compliance_df.to_string(index=False))

In [ ]:
# Query 3: Multi-currency financial consolidation
consolidation_query = """
MATCH (sub:Subsidiary)-[:USES_CURRENCY]->(cur:CurrencyExchange)
WITH sub.incorporation_country AS country,
     cur.base_currency AS local_currency,
     sum(sub.gross_written_premium) AS total_gwp_local,
     cur.exchange_rate AS fx_rate,
     sum(sub.gross_written_premium * cur.exchange_rate) AS total_gwp_usd,
     sum(sub.employee_count) AS total_employees
RETURN country,
       local_currency,
       round(total_gwp_local, 2) AS gwp_local,
       fx_rate,
       round(total_gwp_usd, 2) AS gwp_usd,
       total_employees
ORDER BY gwp_usd DESC
"""

consolidation_df = pd.DataFrame(run_query(consolidation_query))
print("\nMulti-Currency Financial Consolidation:")
print(consolidation_df.to_string(index=False))

# Calculate totals
if not consolidation_df.empty:
    total_gwp_usd = consolidation_df['gwp_usd'].sum()
    total_employees = consolidation_df['total_employees'].sum()
    print(f"\n  Group Total GWP (USD): ${total_gwp_usd:,.2f}")
    print(f"  Group Total Employees: {total_employees}")

## Step 6: Multi-Line Platform Verification

Verify the complete multi-line insurance platform implementation.

In [ ]:
# Complete platform verification
verification_query = """
// Platform verification and node count
MATCH (n)
WITH labels(n)[0] AS node_type, count(n) AS count
ORDER BY count DESC

WITH collect({type: node_type, count: count}) AS node_summary,
     sum(count) AS total_nodes

MATCH ()-[r]->()
WITH node_summary, total_nodes, count(r) AS total_relationships

UNWIND node_summary AS node_info
RETURN node_info.type AS node_type,
       node_info.count AS count,
       total_nodes,
       total_relationships
ORDER BY count DESC
"""

verification_results = run_query(verification_query)
total_nodes = verification_results[0]['total_nodes'] if verification_results else 0
total_relationships = verification_results[0]['total_relationships'] if verification_results else 0

print(f"\n=== Multi-Line Platform Database State ===")
print(f"Total Nodes: {total_nodes}")
print(f"Total Relationships: {total_relationships}")
print("\nNode Distribution:")
for record in verification_results:
    print(f"  {record['node_type']}: {record['count']} nodes")

In [ ]:
# Query product line distribution
product_line_query = """
MATCH (prod:Product)
WITH prod.product_type AS product_line,
     count(*) AS product_count,
     collect(prod.product_name) AS products
RETURN product_line,
       product_count,
       products
ORDER BY product_count DESC
"""

product_line_df = pd.DataFrame(run_query(product_line_query))
print("\nProduct Line Distribution:")
print(product_line_df.to_string(index=False))

In [ ]:
# Query customer type distribution
customer_type_query = """
MATCH (c:Customer)
WITH CASE
       WHEN 'Person' IN labels(c) THEN 'Individual'
       WHEN 'Business' IN labels(c) THEN 'Business'
       ELSE 'Other'
     END AS customer_type,
     count(*) AS customer_count
RETURN customer_type,
       customer_count
ORDER BY customer_count DESC
"""

customer_type_df = pd.DataFrame(run_query(customer_type_query))
print("\nCustomer Type Distribution:")
print(customer_type_df.to_string(index=False))

## Step 7: Generate Platform Summary Report

In [ ]:
# Generate comprehensive platform summary
def generate_platform_summary():
    print("="*70)
    print("MULTI-LINE INSURANCE PLATFORM SUMMARY")
    print("="*70)
    
    print(f"\nDATABASE METRICS:")
    print(f"   Total Nodes: {total_nodes}")
    print(f"   Total Relationships: {total_relationships}")
    print(f"   Database Growth: Comprehensive multi-line platform from Lab 15")
    
    print(f"\nINSURANCE PRODUCT LINES:")
    print(f"   Personal Lines: Auto & Property Insurance")
    print(f"   Life Insurance: Term Life & Whole Life Products")
    print(f"   Commercial Lines: General Liability, Workers Comp, Cyber")
    print(f"   Specialty Products: Professional Liability & Umbrella")
    
    print(f"\nBUSINESS OPERATIONS:")
    print(f"   Individual Customers: Personal insurance policyholders")
    print(f"   Business Customers: Commercial insurance accounts")
    print(f"   Life Insurance: Beneficiary management & cash values")
    print(f"   Claims Management: Multi-line claims processing")
    
    print(f"\nGLOBAL OPERATIONS:")
    print(f"   US Operations: Primary market with state compliance")
    print(f"   UK Subsidiary: FCA regulated with Solvency II compliance")
    print(f"   Canada Subsidiary: OSFI regulated with MCCSR requirements")
    print(f"   Multi-Currency: USD, GBP, CAD with real-time exchange rates")
    
    print(f"\nREINSURANCE NETWORKS:")
    print(f"   Munich Re America: Traditional reinsurance partner")
    print(f"   Swiss Re Corporate: Cyber & specialty coverage")
    print(f"   Berkshire Hathaway: Catastrophe & large risk coverage")
    print(f"   Treaty Management: Quota Share, Excess of Loss, Specialty")
    
    print(f"\nREGULATORY COMPLIANCE:")
    print(f"   US Compliance: State insurance regulation frameworks")
    print(f"   UK Compliance: FCA/PRA supervision under Solvency II")
    print(f"   Canada Compliance: OSFI supervision with provincial licensing")
    print(f"   Reporting: Quarterly regulatory reporting across jurisdictions")
    
    print(f"\nENTERPRISE FEATURES:")
    print(f"   Multi-Jurisdiction Operations: Seamless cross-border management")
    print(f"   Currency Management: Automated USD conversion capabilities")
    print(f"   Risk Distribution: Comprehensive reinsurance risk transfer")
    print(f"   Product Portfolio: Complete insurance value chain coverage")
    
    print(f"\nNEXT PHASE READINESS:")
    print(f"   1. AI/ML Integration: Predictive analytics & automated underwriting")
    print(f"   2. IoT Data Streams: Telematics & smart device integration")
    print(f"   3. Blockchain Technology: Smart contracts & parametric insurance")
    print(f"   4. Advanced Visualization: 3D network displays & real-time analytics")
    print(f"   5. Real-time Streaming: Live data processing & instant risk assessment")
    
    print(f"\nCOURSE PROGRESSION:")
    print(f"   Lab 15: Multi-Line Insurance Platform - COMPLETED")
    print(f"   Next: Lab 17 - Innovation Showcase & Future Capabilities")
    print(f"   Ready for cutting-edge InsurTech innovation")
    
    print("="*70)

# Generate final summary
generate_platform_summary()

print("\nCONGRATULATIONS!")
print("You have successfully built a comprehensive multi-line insurance platform")
print("that rivals enterprise systems used by major global insurance companies!")
print("\nReady for Lab 17: Innovation Showcase & Future Capabilities")

## Step 8: Summary

### What You've Accomplished:

**International Subsidiaries:**
- Created UK subsidiary with FCA/PRA regulatory oversight
- Created Canada subsidiary with OSFI federal regulation
- Implemented Solvency II compliance framework (UK)
- Implemented MCCSR capital adequacy requirements (Canada)
- Established provincial licensing structure for Canadian operations

**Multi-Currency Operations:**
- Built currency exchange rate system (USD, GBP, CAD)
- Implemented bid-ask spreads for realistic FX trading
- Linked subsidiaries to primary currencies
- Created financial consolidation capability to USD

**Regulatory Compliance:**
- UK: Solvency II, SMCR, UK GDPR, ICOBS/COBS
- Canada: OSFI guidelines, PIPEDA, provincial regulations
- Quarterly reporting requirements across jurisdictions
- Capital adequacy monitoring and solvency ratios

**Complete Multi-Line Platform:**
- Personal Lines: Auto and property insurance
- Life Insurance: Term and whole life with beneficiaries
- Commercial Lines: GL, workers comp, cyber liability
- Specialty Products: Professional liability, umbrella
- Reinsurance: Global partnerships and treaty management
- Global Operations: Multi-country, multi-currency platform

### Enterprise Insurance Platform Capabilities:

1. **Complete Product Portfolio**: Spanning all major insurance lines
2. **Global Scalability**: Multi-country operations with regulatory compliance
3. **Risk Management**: Comprehensive reinsurance networks and treaty structures
4. **Operational Efficiency**: Integrated systems supporting complete value chain
5. **Financial Consolidation**: Multi-currency with automated USD conversion
6. **Regulatory Compliance**: Framework supporting multiple jurisdictions

---

**CONGRATULATIONS!** You have completed Lab 15 and built a world-class multi-line insurance platform!

This platform includes:
- Multiple insurance product lines (personal, commercial, life, specialty)
- Global operations across three countries
- Comprehensive reinsurance networks
- Multi-currency financial management
- Regulatory compliance frameworks
- Complex customer and policy management

You are now ready for **Lab 17: Innovation Showcase & Future Capabilities**!